# Лабораторная работа 3
Представлены два набора данных. Необходимо проверить гипотезы об однородности вида смещение и сжатие/растяжение против альтернатив о том, что между выборки имеется различие.
    
Для решения задачи применить корректную процедуру, используя наиболее подходящие критерии (согласно их мощности и предположениям).

Уровень значимости принять равным 0,05.
    
Итоговый файл **.csv** должен содержать 6 чисел, через запятую: 

* значение статистики при проверке гипотезы об однородности вида смещение
* p-значение статистики при проверке гипотезы об однородности вида смещение
* итоговая гипотеза при проверке гипотезы об однородности вида смещение (0 или 1)
* значение статистики при проверке гипотезы об однородности вида сжатие/растяжение
* p-значение статистики при проверке гипотезы об однородности вида сжатие/растяжение
* итоговая гипотеза при проверке гипотезы об однородности вида сжатие/растяжение (0 или 1)

In [1]:
from math import sqrt
from pathlib import Path
from typing import List, Any, Tuple

import numpy as np

from scipy import stats
import pandas as pd

In [2]:
file_path = Path('.')
data_path = file_path / '..' / 'data' / 'lw3' / 'projects' / '1032173841'
data_path = data_path.resolve()

s1 = np.array([row.item() for _, row in pd.read_csv(data_path/'sample1.csv', header=None).iterrows()])
     
s2 = np.array([row.item() for _, row in pd.read_csv(data_path/'sample2.csv', header=None).iterrows()])

alpha = 0.05

In [3]:
def mean(lst: List[float]) -> float:
    return sum(lst)/len(lst)

def var(lst: List[float]) -> float:
    var = (
        sum([(l_i - mean(lst))**2 for l_i in lst]) / 
    #   ------------------------------------------
                         (len(lst)-1)
    )
    return var

def rank(lst: List[float]) -> List[float]:
    _srt: List[Tuple[float, int]] = list(enumerate(sorted(lst), 1))
    srt = [[el[0], el[1]] for el in _srt]
    
    tmp: int = 0
    for idx, val in srt:
        if val == srt[idx-2][1]:
            tmp += 1
        else:
            if tmp != 0:
                new_idx = mean([el[0] for el in srt[idx-2-tmp:idx-1]])
                for i in range(idx-2-tmp, idx-1):
                    srt[i][0] = new_idx
                tmp = 0
    if tmp != 0:
        new_idx = mean([el[0] for el in srt[idx-1-tmp:idx]])
        for i in range(idx-1-tmp, idx):
            srt[i][0] = new_idx
        tmp = 0
    
    mp: Dict[float, float] = {}
    
    for idx, val in srt:
        if val not in mp:
            mp[val] = idx
    
    res: List[int] = [mp[el] for el in lst]
    return res

In [4]:
round(stats.tvar(s1), 5), round(var(s1), 5), round(stats.tvar(s1), 5) == round(var(s1), 5)

(8546.662, 8546.662, True)

In [5]:
tst = [0, 15, 3, 15, 15, 15, 3, 12]
r = rank(tst)
print(r, stats.rankdata(tst), r == stats.rankdata(tst))

[1, 6.5, 2.5, 6.5, 6.5, 6.5, 2.5, 4] [1.  6.5 2.5 6.5 6.5 6.5 2.5 4. ] [ True  True  True  True  True  True  True  True]


#### 1: check for normality

In [6]:
s1_norm_s, s1_norm_p = stats.shapiro(s1)
s2_norm_s, s2_norm_p = stats.shapiro(s2)
if (s1_norm_p > alpha) and (s2_norm_s > alpha):
    print("confirm H0 for both samples (normal distribution)")
else:
    print("reject  H0 for both samples")

confirm H0 for both samples (normal distribution)


#### samples have normal distribution, so we need to use Fisher test

In [7]:
def fisher_test(x: List[float], y: List[float]) -> Tuple[float, float]:
    statistic = var(s1) / var(s2)
    p_value = 1 - stats.f.cdf(statistic, x.size - 1, y.size - 1)
    return statistic, p_value

print(fisher_test(s1, s2))

(0.9816459432939638, 0.5319786290765001)


In [8]:
rast_s, rast_p = fisher_test(s2, s1)
if rast_p >= alpha:
    rast_h = 0
    print("confirm H0")
else:
    rast_h = 1
    print("reject  H0")

confirm H0


#### H0 confirmed -> vars are equal -> Student(t-test)

#### student

In [9]:
def student(x: List[float], y: List[float]) -> Tuple[float, float]:
    x_mean: float = mean(x)
    y_mean: float = mean(y)
    
    s1: float = sum([(x_i - x_mean) ** 2 for x_i in x])
    s2: float = sum([(y_i - y_mean) ** 2 for y_i in y])
    
    s_p = np.sqrt((s1 + s2) / (x.size + y.size - 2))
    
    statistic = (x_mean - y_mean) / (s_p * np.sqrt(1/y.size + 1/x.size))
    p_value = 2 * (1 - stats.t.cdf(np.abs(statistic), x.size + y.size - 2))
    return statistic, p_value

In [10]:
stats_odno_s, stats_odno_p = stats.ttest_ind(s1, s2)
odno_s, odno_p = student(s1, s2)
if (round(stats_odno_s, 5) == round(odno_s, 5)) and (round(stats_odno_p, 5) == round(odno_p, 5)):
    print("realization same as in stats")
print(odno_s, odno_p)
if odno_p >= alpha:
    odno_h = 0
    print("confirm H0")
else:
    odno_h = 1
    print("reject H0")

realization same as in stats
0.9337511590444215 0.35190354593710804
confirm H0


In [11]:
data = {
    'odno_s': [odno_s],
    'odno_p': [odno_p],
    'odno_h': [odno_h],
    'rast_s': [rast_s],
    'rast_p': [rast_p],
    'rast_h': [rast_h],
}

df = pd.DataFrame(data=data)
display(df)

csv_path = Path('.') / 'lab03.csv'
csv_path = csv_path.resolve()

if csv_path.exists():
    csv_path.unlink()

df.to_csv(csv_path, index=False)

,odno_s,odno_p,odno_h,rast_s,rast_p,rast_h
0,0.933751,0.351904,0,0.981646,0.532377,0
